In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import  LinearRegression
from Data_split import data_split, pd_ret_split, R2_score
import math
from tqdm import tqdm

# Perform PCR to full data

In [ ]:

dt = pd.read_csv('merge_data.csv')
dt



    

In [4]:
ncom_list = [ ]
R2_list = [ ]

In [ ]:

for i in tqdm(range(199001,202001,100)):
    
    train,val,test = data_split(i, dt)
    x_train, y_train = pd_ret_split(train)
    x_val, y_val = pd_ret_split(val)
    x_test, y_test = pd_ret_split(test)
    
    best_valR2 = float("-Inf")
    best_pcr = None
    
                         
    for K in range(5,50,10):
        pca = PCA(n_components = K)
        pcr = make_pipeline(StandardScaler(), pca, LinearRegression())
        pcr.fit(x_train,y_train)
    
        val_pred = pcr.predict(x_val)
        cval_R2 = R2_score(y_val, val_pred) #annual R2 score
            
        if(cval_R2 > best_valR2):
            best_valR2 = cval_R2
            best_pcr = pcr
            best_ncom = K

        
    test_pred = best_pcr.predict(x_test)
    testR2 = R2_score(y_test, test_pred)

    ncom_list.append(best_ncom)
    R2_list.append(testR2)

In [227]:
ncom_list = pd.DataFrame(ncom_list) 
R2_list = pd.DataFrame(R2_list)

In [242]:
best_year=np.argmax(R2_list)
best_k = ncom_list.iloc[best_year,0]
best_year= 199001+best_year*100
best_R2 = np.max(R2_list)
best_year,best_k, best_R2
    
   
    

# Analyze variable importance of PCR

    From above, we attain the testing year with best R2 and best component number. Based on this PCR model, we measure each predictors's importance using the R2 reduction from setting all values of this specified predictor to zero, while hoilding the remaining model estimates fiexed.

In [3]:
var = dt.columns

In [4]:
var = var.drop(['permno','yyyymm','excess_ret'])

In [ ]:
var

In [ ]:
var_result = list()
sum = 0
for j in tqdm(var):
    dt_pd = dt.copy()
    dt_pd[j]=0
  
        
         
    train,val,test = data_split(best_year, dt_pd)
    x_train, y_train = pd_ret_split(train)
    x_test, y_test = pd_ret_split(test)
    #y_test = np.array(y_test)
    #y_test = y_test.reshape(len(y_test),1)

    pca = PCA(n_components = best_k)
    pcr = make_pipeline(StandardScaler(), pca, LinearRegression())
    pcr.fit(x_train,y_train)
    
    test_pred = pcr.predict(x_test)
    testR2 = R2_score(y_test, test_pred)
    
  
    Im = abs(best_R2 - testR2)
    #Im = Im.to_numpy()
    sum = sum +Im
    var_result.append([j, Im])

In [ ]:
for i in range(0,len(var_result)):
    var_result[i][1] = var_result[i][1]/sum

var_result = pd.DataFrame(var_result,columns = ['Variable','Importance'])



In [ ]:
var_result = var_result.sort_values(by = 'Importance', ascending = False)
var_result.head(20)